In [1]:
import pandas as pd

Eu_df = pd.read_csv('TSP_1000_euclidianDistance.txt',sep=r'\s+', header=1)
Ran_df = pd.read_csv('TSP_1000_randomDistance.txt',sep=r'\s+', header=1)

print(Eu_df.head())

   Node1  Node2  Distance
0      1      2     26.98
1      1      3     53.74
2      1      4     57.68
3      1      5     29.43
4      1      6     27.00


In [2]:
import networkx as nx
import numpy as np

G1 = nx.Graph()
for _, row in Eu_df.iterrows():
    G1.add_edge(int(row['Node1']), int(row['Node2']), weight=int(100*row['Distance']))
G2 = nx.Graph()
for _, row in Ran_df.iterrows():
    G2.add_edge(int(row['Node1']), int(row['Node2']), weight=int(100*row['Distance']))

In [3]:
def greedy_tsp_cycle(graph):
    """贪心算法求最短哈密顿回路"""
    nodes = list(graph.nodes())
    start = nodes[0]
    
    visited = [start]
    current = start
    total_distance = 0
    loop_count = 0 

    
    while len(visited) < len(nodes):

        min_dist = float('inf')
        next_node = None
        
        for node in nodes:
            if node not in visited and graph.has_edge(current, node):
                loop_count += 1  # ← 每次循环加1
                dist = graph[current][node]['weight']
                if dist < min_dist:
                    min_dist = dist
                    next_node = node
        
        if next_node is None:
            return None, float('inf')  # 无法完成回路
            
        visited.append(next_node)
        total_distance += min_dist
        current = next_node
    
    # 回到起点形成回路
    if graph.has_edge(current, start):
        total_distance += graph[current][start]['weight']
        visited.append(start)
    else:
        return None, float('inf')
    
    return visited, total_distance,loop_count

path_g1, distance_g1,loop_g1 = greedy_tsp_cycle(G1)
print(f"最短回路总距离: {float(distance_g1/100)}；循环数量{loop_g1}")
print(f"路径: {path_g1[:100]}...（共{len(path_g1)}个节点）")

最短回路总距离: 2969.16；循环数量499500
路径: [1, 303, 675, 426, 29, 555, 184, 397, 850, 514, 75, 969, 910, 380, 44, 496, 900, 500, 526, 525, 711, 243, 907, 27, 510, 814, 751, 47, 449, 575, 216, 646, 443, 957, 197, 585, 754, 177, 705, 976, 922, 450, 898, 610, 23, 859, 862, 371, 134, 69, 647, 194, 746, 191, 812, 9, 198, 454, 702, 444, 802, 368, 160, 236, 956, 425, 351, 628, 594, 509, 54, 479, 764, 538, 680, 238, 698, 325, 147, 637, 219, 30, 283, 939, 150, 268, 942, 935, 877, 35, 378, 232, 709, 26, 790, 81, 272, 818, 281, 638]...（共1001个节点）


In [4]:
path_g2, distance_g2,loop_g2 = greedy_tsp_cycle(G2)
print(f"最短回路总距离: {float(distance_g2/100)}循环数量{loop_g2}")
print(f"路径: {path_g2[:10]}...（共{len(path_g2)}个节点）")

最短回路总距离: 667.63循环数量499500
路径: [1, 995, 112, 404, 240, 682, 908, 156, 711, 530]...（共1001个节点）


In [6]:
import random
import time

def tabu_search_tsp(graph, initial_path=None, time_limit=60, tabu_size=100):
    """禁忌搜索算法（可指定初始解）"""
    
    nodes = list(graph.nodes())
    n = len(nodes)
    
    # 预计算距离矩阵
    dist_matrix = {}
    for u in nodes:
        for v in graph.neighbors(u):
            dist_matrix[(u, v)] = graph[u][v]['weight']
    
    def get_dist(u, v):
        return dist_matrix.get((u, v), float('inf'))
    
    def calculate_distance(path):
        return sum(get_dist(path[i], path[i+1]) for i in range(len(path)-1))
    
    # ===== 使用贪心解作为初始解 =====
    if initial_path:
        current_path = initial_path.copy()
    else:
        current_path = nodes.copy()
        random.shuffle(current_path)
        current_path.append(current_path[0])
    
    current_dist = calculate_distance(current_path)
    best_path = current_path.copy()
    best_dist = current_dist
    initial_dist = current_dist
    
    tabu_list = []
    evaluations = 0
    iterations = 0
    improvements = 0
    
    start_time = time.time()
    last_print_time = start_time
    
    
    while True:
        if time.time() - start_time >= time_limit:
            print("  时间到！")
            break
        
        iterations += 1
        best_neighbor = None
        best_neighbor_dist = float('inf')
        best_move = None
        
        # 探索邻域（2-opt）
        for i in range(1, n - 1):
            for j in range(i + 1, n):
                evaluations += 1
                move = (i, j)
                
                # 增量计算距离变化
                old_cost = get_dist(current_path[i-1], current_path[i]) + get_dist(current_path[j], current_path[j+1])
                new_cost = get_dist(current_path[i-1], current_path[j]) + get_dist(current_path[i], current_path[j+1])
                new_dist = current_dist + (new_cost - old_cost)
                
                # 检查是否在禁忌表中（藐视准则：如果是最优解则忽略禁忌）
                if move not in tabu_list or new_dist < best_dist:
                    if new_dist < best_neighbor_dist:
                        best_neighbor_dist = new_dist
                        best_move = move
        
        if best_move:
            i, j = best_move
            # 执行2-opt交换
            current_path = current_path[:i] + current_path[i:j+1][::-1] + current_path[j+1:]
            current_dist = best_neighbor_dist
            
            # 更新禁忌表
            tabu_list.append(best_move)
            if len(tabu_list) > tabu_size:
                tabu_list.pop(0)
            
            # 更新最优解
            if current_dist < best_dist:
                best_path = current_path.copy()
                best_dist = current_dist
                improvements += 1
        
        # 每10秒显示进度
        if time.time() - last_print_time >= 10:
            print(f"    [{int(time.time() - start_time)}秒] 迭代: {iterations}, 最优: {round(best_dist, 2)}")
            last_print_time = time.time()
    
    return {
        'path': best_path,
        'distance': best_dist,
        'initial_distance': initial_dist,
        'evaluations': evaluations,
        'iterations': iterations,
        'improvements': improvements,
        'time': round(time.time() - start_time, 2)
    }

print("=" * 60)
print("G1")

# 禁忌搜索（基于贪心解）
print(f"\n【禁忌搜索】")
result_tabu_g1 = tabu_search_tsp(G1, initial_path=path_g1, time_limit=60)
print(f"  FinalDistance: {round(result_tabu_g1['distance'], 2)}")
print(f"  Inprovement: {result_tabu_g1['improvements']}")

print("\n" + "=" * 60)
print("G2（随机距离）")
print("=" * 60)

# 禁忌搜索（基于贪心解）
print(f"\n【禁忌搜索】")
result_tabu_g2 = tabu_search_tsp(G2, initial_path=path_g2, time_limit=60)
print(f"  FinalDistance: {round(result_tabu_g2['distance'], 2)}")
print(f"  Inprovement: {result_tabu_g2['improvements']}")

# ========================================
# 总结对比
# ========================================
print("\n" + "=" * 60)
print("总结对比")
print("=" * 60)
print(f"{'图':<6} {'贪心距离':<12} {'禁忌距离':<12} {'改进幅度':<10} {'迭代次数':<10} {'评估次数':<12}")
print("-" * 60)
imp_g1 = (distance_g1 - result_tabu_g1['distance']) / distance_g1 * 100
imp_g2 = (distance_g2 - result_tabu_g2['distance']) / distance_g2 * 100
print(f"{'G1':<6} {round(distance_g1, 2):<12} {round(result_tabu_g1['distance'], 2):<12} {round(imp_g1, 2)}%{'':<6} {result_tabu_g1['iterations']:<10} {result_tabu_g1['evaluations']:<12}")
print(f"{'G2':<6} {round(distance_g2, 2):<12} {round(result_tabu_g2['distance'], 2):<12} {round(imp_g2, 2)}%{'':<6} {result_tabu_g2['iterations']:<10} {result_tabu_g2['evaluations']:<12}")

G1

【禁忌搜索】
    [10秒] 迭代: 14, 最优: 277054
    [21秒] 迭代: 26, 最优: 269653
    [32秒] 迭代: 37, 最优: 264608
    [43秒] 迭代: 47, 最优: 261279
    [53秒] 迭代: 56, 最优: 258667
  时间到！
  FinalDistance: 257218
  Inprovement: 61

G2（随机距离）

【禁忌搜索】
    [10秒] 迭代: 14, 最优: 41406
    [21秒] 迭代: 26, 最优: 38691
    [32秒] 迭代: 37, 最优: 37316
    [42秒] 迭代: 47, 最优: 36815
    [53秒] 迭代: 56, 最优: 36649
  时间到！
  FinalDistance: 36287
  Inprovement: 57

总结对比
图      贪心距离         禁忌距离         改进幅度       迭代次数       评估次数        
------------------------------------------------------------
G1     296916       257218       13.37%       61         30408561    
G2     66763        36287        45.65%       62         30907062    
